# Chapter5
Here we look at what happens when we project optimized cells onto a druckman feature space.

Models, and real cell recordings are seperable in a high dimensional feature space.

We want to use this information, to see if data specific optimized models are better mimics of real cells, than other parameterizations of models

In [19]:
import os
import pickle
try:
    import nbimporter
except:
    !pip install nbimporter
    import nbimporter
from importlib import reload
result_RAW = pickle.load(open('RAWall_data_tests.p','rb'))
result_ADEXP = pickle.load(open('ADEXPall_data_tests.p','rb'))
result_RAW = result_RAW['RAW']

In [20]:

def result_to_dict(result_ADEXP,result_RAW):
    '''
    Reformat dictionaries
    
    '''
    RAW_dtc = {}
    ADEXP_dtc = {}
    RAW_dtc = {}
    try:
        RAW_dtc['Cerebellum Purkinje cell'] = result_RAW['purkine']
    except:
        RAW_dtc['Cerebellum Purkinje cell'] = result_RAW['purkinje']        
    RAW_dtc['Olfactory bulb (main) mitral cell'] = result_RAW['olf']
    RAW_dtc['Hippocampus CA1 pyramidal cell'] = result_RAW['ca1pyr']
    RAW_dtc['Neocortex pyramidal cell layer 5-6'] = result_RAW['neo']
    RAW_dtc['Hippocampus CA1 basket cell'] = result_RAW['ca1basket']
    ADEXP_dtc = {}
    try:
        ADEXP_dtc['Cerebellum Purkinje cell'] = [d.dtc for d in result_ADEXP['ADEXP']['purkine']]
    except:
        ADEXP_dtc['Cerebellum Purkinje cell'] = [d.dtc for d in result_ADEXP['ADEXP']['purkinje']]        
    ADEXP_dtc['Olfactory bulb (main) mitral cell'] = [d.dtc for d in result_ADEXP['ADEXP']['olf']]
    ADEXP_dtc['Hippocampus CA1 pyramidal cell'] = [d.dtc for d in result_ADEXP['ADEXP']['ca1pyr']]
    ADEXP_dtc['Neocortex pyramidal cell layer 5-6'] = [d.dtc for d in result_ADEXP['ADEXP']['neo']]
    ADEXP_dtc['Hippocampus CA1 basket cell'] = [d.dtc for d in result_ADEXP['ADEXP']['ca1basket']]
    return ADEXP_dtc,RAW_dtc

ADEXP_dtc,RAW_dtc = result_to_dict(result_ADEXP,result_RAW)

#RAW_dtc

In [21]:
for cell in RAW_dtc.keys():
    compare0 = {t.name:t.observation for t in RAW_dtc[cell][0].tests.values() }
    compare0 = {k:(v,RAW_dtc[cell][0].predictions[k]) for k,v in compare0.items() }

    compare1 = {t.name:t.observation for t in ADEXP_dtc[cell][0].tests.values() }
    #print(compare1.keys())
    #print(ADEXP_dtc[cell][0].predictions.keys())
    compare1 = {k:(v,ADEXP_dtc[cell][0].predictions[k]) for k,v in compare1.items() if k in ADEXP_dtc[cell][0].predictions.keys()}



In [22]:
pre_df = {}

import chapter1 as chapter1

df6 = chapter1.format_nice_frame(compare0)
df6

,CapacitanceTest,InjectedCurrentAPAmplitudeTest,InjectedCurrentAPThresholdTest,InjectedCurrentAPWidthTest,InputResistanceTest,RestingPotentialTest,RheobaseTest,TimeConstantTest
0,"(4.450000000000001e-11 s**4*A**2/(kg*m**2), 1....","(0.05702 V, 0.05005971458960177 V, 0.855300000...","(-38.72 mV, -0.00774734639432745 mV, -580.8 mV)","(0.0004 s, 0.00055 s, 0.006 s)","(69700000.0 kg*m**2/(s**3*A**2), 6738105036.81...","(-0.06425 V, -0.056 V, -0.96375 V)","(50.0 pA, 0.1910277762192436 pA, 750.0 pA)","(0.009000000000000001 s, 0.0008107699250056406..."


In [23]:
df7 = chapter1.format_nice_frame(compare1)#compare1
df7.T

,0
CapacitanceTest,"(4.450000000000001e-11 s**4*A**2/(kg*m**2), -7..."
InputResistanceTest,"(69700000.0 kg*m**2/(s**3*A**2), 378830061.774..."
RestingPotentialTest,"(-0.06425 V, -0.06999942989522692 V, 0.0065160..."
TimeConstantTest,"(0.009000000000000001 s, 0.0034448357305207145..."


# In notebook cells below we compute Druckman and Allen SDK
Features on the optimized cells, in order to see if the optimized cells fall into pre-defined clusters, differentally  associated with experimental data, and model output data.

To say this another way, it has previously been observed that experimental cell data, and model data, falls into easily seperated categories in feature space. This probably reflects deficits in simple model realism.

By using optimized cells as new data points, and plotting their positions in a reduced dimension feature space we will be able to see if optimized models, are more convincing imitations of experimental data, by testing if new data points are harder to seperate from the experimental data.
### Get Druckman features using a parallel algorithm to save time

In [24]:
#ADEXP_dtc

In [33]:
import dask.bag as db
from neuronunit.optimisation.optimization_management import inject_and_plot, nuunit_dm_evaluation 
dfeatures={}
druckman_feature_coordinatesadexp = {}
for k,v in ADEXP_dtc.items():
    bagged = db.from_sequence([ADEXP_dtc[k][0]])
    druckman_feature_coordinatesadexp[k] = list(bagged.map(nuunit_dm_evaluation).compute())
    dfeatures[k] = [d.dm_test_features for d in  druckman_feature_coordinatesadexp[k] ]


In [30]:
import pandas as pd
druck1 = pd.DataFrame([dfeatures[k][0]])
druck1.T


,0
AP12AHPDepthPercentChangeTest,[0.0 dimensionless]
AP12AmplitudeChangePercentTest,None
AP12AmplitudeDropTest,[0.0 V]
AP12HalfWidthChangePercentTest,None
AP12RateOfChangePeakToTroughPercentChangeTest,-0.0 dimensionless
AP1AHPDepthTest,[0.08956980962528577 V]
AP1AmplitudeTest,None
AP1DelayMeanStrongStimTest,2.9999999999998863 ms
AP1DelayMeanTest,8.0 ms
AP1DelaySDStrongStimTest,0.0 ms


In [36]:
import dask.bag as db
izhidfeatures={}
df_container = {}
druckman_feature_coordinates_izhi ={}
for k,v in RAW_dtc.items():
    bagged = db.from_sequence([RAW_dtc[k][0]])
    druckman_feature_coordinates_izhi[k] = list(bagged.map(nuunit_dm_evaluation).compute())
    izhidfeatures[k] = [d.dm_test_features for d in druckman_feature_coordinates_izhi[k] ]
    druck0 = pd.DataFrame([izhidfeatures[k][0]])
    druck0 = druck0.rename(index={0: str('Izhikivitch')})
    druck1 = dfeatures[k]
    druck1 = druck1.rename(index={0: str('ADEXP')})
    
    #druck4 = pd.DataFrame([izhidfeatures[k][-1]])
    #druck4 = druck4.rename(index={0: str('Izhi_last')})
    #druck3 = pd.DataFrame([dfeatures[k][-1]])
    #druck3 = druck3.rename(index={0: str('ADEXP_last')})
    #druck2 = druck1
    druck2 = druck1.append(druck0)
    #druck2 = druck2.append(druck3)
    #druck2 = druck2.append(druck4)
    df_container[k] = druck2
df_container[k]    

AttributeError: 'list' object has no attribute 'rename'

In [ ]:
cwd = os.getcwd()
import numpy as np
# Open the 1.5x rheobase file
filename = os.path.join(cwd,'onefive_df.pkl')
with open(filename, 'rb') as f:
    df = pickle.load(f)
    
df.iloc[0][192:195]

In [35]:
#RAW_dtc = [d.dtc for d in RAW[k]]

from neuronunit.optimisation.optimization_management import just_allen_predictions
RAW_dtc = [d.dtc for d in RAW[k]]
RAW_dtc[0].protocols[1].keys()

RAW_dtc[0].ampl = RAW_dtc[0].protocols[1]['injected_square_current']['amplitude']*1.5
RAW_dtc[0].ampl
#for i in RAW_dtc:
    
def cell_to_allen(dtc):    
    dtc.pre_obs = None
    dtc.ampl = dtc.protocols[1]['injected_square_current']['amplitude']*3.0
    dtc = just_allen_predictions(dtc)
    return dtc
bagged = db.from_sequence(RAW_dtc)
RAW_dtc = list(bagged.map(cell_to_allen).compute())

NameError: name 'RAW' is not defined

In [ ]:
preds = RAW_dtc[1][0].preds
RAW_dtc[1][0].preds.pop('spikes',None)
preds = {k:v['mean'] for k,v in RAW_dtc[1][0].preds.items()}
preds
allen_df = pd.DataFrame([preds])
allen_df.T

